In [1]:
from pyspark import SQLContext
from pyspark import HiveContext
from pyspark.sql import Row
df = spark.read.format('csv').load ('dbfs:/FileStore/tables/employeetable.csv',header = True)
df.show()

+------+------------+----------+------+
empid | empname | location |salary|
+------+------------+----------+------+
 100|karthikeyan | chennai| 1003|
 200| navaneethan|coimbatore| 2004|
 300| nalini| madurai| 3005|
 400| prakash|coimbatore| 4006|
 500| nivedhithaa| dindugal| 5007|
+------+------------+----------+------+

In [2]:
df.count()

Out[ 8 ]: 5

In [3]:
df.select('location ').count()

Out[ 11 ]: 5

In [4]:
from pyspark.sql.functions import countDistinct 
df.select(countDistinct("location ")).show()

+-------------------------+
count(DISTINCT location )|
+-------------------------+
 4|
+-------------------------+

In [5]:
#approx_count_distinct
from pyspark.sql.functions import approx_count_distinct
df.select(approx_count_distinct("location ")).show()

+--------------------------------+
approx_count_distinct(location )|
+--------------------------------+
 3|
+--------------------------------+

In [6]:
#first,last
from pyspark.sql.functions import first,last
df.select(first('location '),last('location ')).show()

+-----------------------+----------------------+
first(location , false)|last(location , false)|
+-----------------------+----------------------+
 chennai| dindugal|
+-----------------------+----------------------+

In [7]:
#min,max
from pyspark.sql.functions import min,max
df.select(min("salary"),max("salary")).show()

+-----------+-----------+
min(salary)|max(salary)|
+-----------+-----------+
 1003| 5007|
+-----------+-----------+

In [8]:
#Suppose If we select non numeric value selected 
from pyspark.sql.functions import min ,max
df.select(min("empname "),max("empname ")).show()

+-------------+-------------+
min(empname )|max(empname )|
+-------------+-------------+
 karthikeyan | prakash|
+-------------+-------------+

In [9]:
#sum 

from pyspark.sql.functions import sum
df.select(sum("salary")).show()

+-----------+
sum(salary)|
+-----------+
 15025.0|
+-----------+

In [10]:
#SumDistinct
from pyspark.sql.functions import sumDistinct
df.select(sumDistinct("salary")).show()

+--------------------+
sum(DISTINCT salary)|
+--------------------+
 15025.0|
+--------------------+

In [11]:
#avg,mean,
from pyspark.sql.functions import avg,sum
df.select(avg("salary"),sum("salary")).show()

+-----------+-----------+
avg(salary)|sum(salary)|
+-----------+-----------+
 3005.0| 15025.0|
+-----------+-----------+

In [12]:
from pyspark.sql.functions import count
df.groupBy("location ")\
  .agg(count("*")).show()

+----------+--------+
 location |count(1)|
+----------+--------+
 chennai| 1|
 dindugal| 1|
 madurai| 1|
coimbatore| 2|
+----------+--------+

In [13]:
# import packages
import os
from pyspark import SparkContext, SparkConf
import numpy as np
# import seaborn as sns
# sns.set(rc={'figure.figsize':(15,5)},font_scale=1.2)
# color = ["#e74c3c", "#2ecc71"]
# sns.set_palette(color)
# sns.set_style("whitegrid")
import matplotlib.pyplot as plt

import pandas as pd
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None)


print(os.environ['SPARK_HOME'])
from pyspark.sql import SparkSession, HiveContext, SQLContext
from pyspark.sql.functions import when, desc
from pyspark.sql.types import *
from pyspark.sql import Row, functions as F
from pyspark.sql.functions import to_date, trunc, unix_timestamp, date_format
from pyspark.sql.window import Window
from datetime import datetime

/databricks/spark

In [14]:
#Window Function 
from pyspark.sql.window import Window
from pyspark.sql.functions import desc
# = Window.partitionBy('store_nbr', 'rx_nbr', 'rx_fill_nbr').orderBy(F.col('run_dt'))
window = Window\
               .partitionBy('location ')\
               .orderBy('salary')
df.show()

+------+------------+----------+------+
empid | empname | location |salary|
+------+------------+----------+------+
 100|karthikeyan | chennai| 1003|
 200| navaneethan|coimbatore| 2004|
 300| nalini| madurai| 3005|
 400| prakash|coimbatore| 4006|
 500| nivedhithaa| dindugal| 5007|
+------+------------+----------+------+

In [15]:
from pyspark.sql.functions import rank
df1 = df.select('location ','salary',rank().over(window).alias('rankcol'))

In [16]:
df1.show()

+----------+------+-------+
 location |salary|rankcol|
+----------+------+-------+
 chennai| 1003| 1|
 dindugal| 5007| 1|
 madurai| 3005| 1|
coimbatore| 2004| 1|
coimbatore| 4006| 2|
+----------+------+-------+

In [17]:
df1.printSchema()

root
-- location : string (nullable = true)
-- salary: string (nullable = true)
-- rankcol: integer (nullable = true)

In [18]:
df1.filter(col("rankcol") == 1).show()

+----------+------+-------+
 location |salary|rankcol|
+----------+------+-------+
 chennai| 1003| 1|
 dindugal| 5007| 1|
 madurai| 3005| 1|
coimbatore| 2004| 1|
+----------+------+-------+

In [19]:
#Grouping set 
df.show()
df.registerTempTable("test")

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1133364376476616> in <module> () 
 1 #Grouping set 
 ----> 2 df . show ( ) 
 3 df . registerTempTable ( "test" ) 

 NameError : name 'df' is not defined

In [20]:
from pyspark import SQLContext
from pyspark import HiveContext
from pyspark.sql import Row
df = spark.read.format('csv').load ('dbfs:/FileStore/tables/employeetable.csv',header = True)
df.show()

+------+------------+----------+------+
empid | empname | location |salary|
+------+------------+----------+------+
 100|karthikeyan | chennai| 1003|
 200| navaneethan|coimbatore| 2004|
 300| nalini| madurai| 3005|
 400| prakash|coimbatore| 4006|
 500| nivedhithaa| dindugal| 5007|
+------+------------+----------+------+

In [21]:
#rollup
from pyspark.sql.functions import count
df.rollup("location ")\
  .agg(count("*")).show()

+----------+--------+
 location |count(1)|
+----------+--------+
 dindugal| 1|
 null| 5|
 madurai| 1|
coimbatore| 2|
 chennai| 1|
+----------+--------+

In [22]:
# cube 
from pyspark.sql.functions import sum
df.cube("location ","empname ")\
  .agg(count("*")).orderBy("location ").show()

+----------+------------+--------+
 location | empname |count(1)|
+----------+------------+--------+
 null| navaneethan| 1|
 null| null| 5|
 null| nalini| 1|
 null|karthikeyan | 1|
 null| prakash| 1|
 null| nivedhithaa| 1|
 chennai|karthikeyan | 1|
 chennai| null| 1|
coimbatore| null| 2|
coimbatore| prakash| 1|
coimbatore| navaneethan| 1|
 dindugal| null| 1|
 dindugal| nivedhithaa| 1|
 madurai| nalini| 1|
 madurai| null| 1|
+----------+------------+--------+

In [23]:
#rollup
from pyspark.sql.functions import count
df_pivoted  = df.groupBy("location ")\
  .pivot("salary").sum()

df_pivoted.show() 

+----------+
 location |
+----------+
 chennai|
 dindugal|
 madurai|
coimbatore|
+----------+

In [24]:
from pyspark import SQLContext
from pyspark import HiveContext
from pyspark.sql import Row
df = spark.read.format('csv').load ('dbfs:/FileStore/tables/employeetable.csv',header = True)
df.show()

+------+------------+----------+------+
empid | empname | location |salary|
+------+------------+----------+------+
 100|karthikeyan | chennai| 1003|
 200| navaneethan|coimbatore| 2004|
 300| nalini| madurai| 3005|
 400| prakash|coimbatore| 4006|
 500| nivedhithaa| dindugal| 5007|
+------+------------+----------+------+

In [25]:
from pyspark.sql import functions as F
df.agg(F.min(df.salary)).collect()
df.agg({'salary':'max'}).collect()

Out[ 7 ]: [Row(max(salary)='5007')]

In [26]:
from pyspark.sql import functions as F
df.agg({'salary':'max'}).collect()

Out[ 9 ]: [Row(max(salary)='5007')]

In [27]:
from pyspark.sql import functions as F
df.agg({'salary':'max'}).collect()

Out[ 10 ]: [Row(max(salary)='5007')]

In [28]:
df.dtypes

Out[ 11 ]: 
[('empid ', 'string'),
 ('empname ', 'string'),
 ('location ', 'string'),
 ('salary', 'string')]

In [29]:
df.schema

Out[ 12 ]: StructType(List(StructField(empid ,StringType,true),StructField(empname ,StringType,true),StructField(location ,StringType,true),StructField(salary,StringType,true)))

In [30]:
from pyspark.sql.functions import col
df = sqlContext.createDataFrame(
    [('cat \n\n elephant rat \n rat cat', )], ['word']
)

df.select(explode(split(col("word"), "\s+")).alias("word")).show()

+--------+
 word|
+--------+
 cat|
elephant|
 rat|
 rat|
 cat|
+--------+

In [31]:
# import pyspark class Row from module sql
from pyspark.sql import *

# Create Example Data - Departments and Employees

# Create the Departments
department1 = Row(id='123456', name='Computer Science')
department2 = Row(id='789012', name='Mechanical Engineering')
department3 = Row(id='345678', name='Theater and Drama')
department4 = Row(id='901234', name='Indoor Recreation')

# Create the Employees
Employee = Row("firstName", "lastName", "email", "salary")
employee1 = Employee('michael', 'armbrust', 'no-reply@berkeley.edu', 100000)
employee2 = Employee('xiangrui', 'meng', 'no-reply@stanford.edu', 120000)
employee3 = Employee('matei', None, 'no-reply@waterloo.edu', 140000)
employee4 = Employee(None, 'wendell', 'no-reply@berkeley.edu', 160000)

# Create the DepartmentWithEmployees instances from Departments and Employees
departmentWithEmployees1 = Row(department=department1, employees=[employee1, employee2])
departmentWithEmployees2 = Row(department=department2, employees=[employee3, employee4])
departmentWithEmployees3 = Row(department=department3, employees=[employee1, employee4])
departmentWithEmployees4 = Row(department=department4, employees=[employee2, employee3])



In [32]:
departmentsWithEmployeesSeq1 = [departmentWithEmployees1, departmentWithEmployees2]
df1 = spark.createDataFrame(departmentsWithEmployeesSeq1)

display(df1)

departmentsWithEmployeesSeq2 = [departmentWithEmployees3, departmentWithEmployees4]
df2 = spark.createDataFrame(departmentsWithEmployeesSeq2)

display(df2)

department,employees
"List(345678, Theater and Drama)","List(List(michael, armbrust, no-reply@berkeley.edu, 100000), List(null, wendell, no-reply@berkeley.edu, 160000))"
"List(901234, Indoor Recreation)","List(List(xiangrui, meng, no-reply@stanford.edu, 120000), List(matei, null, no-reply@waterloo.edu, 140000))"


In [33]:
unionDF = df1.unionAll(df2)
display(unionDF)

department,employees
"List(123456, Computer Science)","List(List(michael, armbrust, no-reply@berkeley.edu, 100000), List(xiangrui, meng, no-reply@stanford.edu, 120000))"
"List(789012, Mechanical Engineering)","List(List(matei, null, no-reply@waterloo.edu, 140000), List(null, wendell, no-reply@berkeley.edu, 160000))"
"List(345678, Theater and Drama)","List(List(michael, armbrust, no-reply@berkeley.edu, 100000), List(null, wendell, no-reply@berkeley.edu, 160000))"
"List(901234, Indoor Recreation)","List(List(xiangrui, meng, no-reply@stanford.edu, 120000), List(matei, null, no-reply@waterloo.edu, 140000))"


In [34]:
df = unionDF.select(explode("employees").alias("e"))

In [35]:
df.show()

+--------------------+
 e|
+--------------------+
[michael, armbrus...|
[xiangrui, meng, ...|
[matei,, no-reply...|
[, wendell, no-re...|
[michael, armbrus...|
[, wendell, no-re...|
[xiangrui, meng, ...|
[matei,, no-reply...|
+--------------------+

In [36]:
explodeDF =df.selectExpr("e.firstname","e.lastname","e.email","e.salary")

In [37]:
explodeDF.show()

+---------+--------+--------------------+------+
firstname|lastname| email|salary|
+---------+--------+--------------------+------+
 michael|armbrust|no-reply@berkeley...|100000|
 xiangrui| meng|no-reply@stanford...|120000|
 matei| null|no-reply@waterloo...|140000|
 null| wendell|no-reply@berkeley...|160000|
 michael|armbrust|no-reply@berkeley...|100000|
 null| wendell|no-reply@berkeley...|160000|
 xiangrui| meng|no-reply@stanford...|120000|
 matei| null|no-reply@waterloo...|140000|
+---------+--------+--------------------+------+

In [38]:
explodeDF.printSchema()

root
-- firstname: string (nullable = true)
-- lastname: string (nullable = true)
-- email: string (nullable = true)
-- salary: long (nullable = true)

In [39]:
filterDF = explodeDF.filter(explodeDF.firstname == "xiangrui").sort(explodeDF.lastname)
display(filterDF)

firstname,lastname,email,salary
xiangrui,meng,no-reply@stanford.edu,120000
xiangrui,meng,no-reply@stanford.edu,120000


In [40]:
df = spark.createDataFrame([(1, "A", [1,2,3]), (2, "B", [3,5])],["col1", "col2", "col3"])
from pyspark.sql.functions import explode
df.show()

+----+----+---------+
col1|col2| col3|
+----+----+---------+
 1| A|[1, 2, 3]|
 2| B| [3, 5]|
+----+----+---------+

In [41]:
df.withColumn("col3", explode(df.col3)).show()

+----+----+----+
col1|col2|col3|
+----+----+----+
 1| A| 1|
 1| A| 2|
 1| A| 3|
 2| B| 3|
 2| B| 5|
+----+----+----+